In [31]:
# -*- coding:utf-8 -*-

from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.neural_network import MLPClassifier
from tflearn.layers.normalization import local_response_normalization
# from tensorflow.contrib import learn
import gensim
import re
from collections import namedtuple
from gensim.models import Doc2Vec
# from gensim.models.doc2vec import Doc2Vec,LabeledSentence
from random import shuffle
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import preprocessing
from hmmlearn import hmm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from tflearn.layers.recurrent import bidirectional_rnn, BasicLSTMCell

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


# 14.1　数据集

In [4]:
def load_files_from_dir(dir):
    import glob
    files=glob.glob(dir)
    result = []
    for file in files:
        print("Load file %s" % file)
        with open(file) as f:
            lines=f.readlines()
            lines_to_line=" ".join(lines)
            lines_to_line = re.sub(r"[APT|Crypto|Locker|Zeus]", ' ', lines_to_line,flags=re.I)
            result.append(lines_to_line)
    return result

In [33]:
import glob
import os
import re

def load_files_from_dir(dir_path):
    # 确保路径是绝对的
    dir_path = os.path.abspath(dir_path)
    dir_pattern = os.path.join(dir_path, '*')
    files = glob.glob(dir_pattern)
    
    result = []
    for file in files:
        # print("Load file %s" % file)
        with open(file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            lines_to_line = " ".join(lines)
            lines_to_line = re.sub(r"[APT|Crypto|Locker|Zeus]", ' ', lines_to_line, flags=re.IGNORECASE)
            result.append(lines_to_line)
    return result

def load_files():
    base_dir = os.path.join('..', 'Data', 'MalwareTrainingSets-master', 'trainingSets')
    malware_class = ['APT1', 'Crypto', 'Locker', 'Zeus']
    x = []
    y = []
    for i, family in enumerate(malware_class):
        dir_path = os.path.join(base_dir, family)  # 构建完整的目录路径
        # print("Load files from %s index %d" % (dir_path, i))
        v = load_files_from_dir(dir_path)
        x += v
        y += [i] * len(v)
    print("Loaded files %d" % len(x))
    return x, y

# 调用函数
x, y = load_files()

Loaded files 4762


In [22]:
import pandas as pd

df = pd.DataFrame({'x':x,'y':y})
df

,x,y
0,"{"" n i Id"": 652, "" n i "": "" n n "", "" v...",0
1,"{"" n i Id"": 653, "" n i "": "" n n "", "" v...",0
2,"{"" n i Id"": 654, "" n i "": "" n n "", "" v...",0
3,"{"" n i Id"": 655, "" n i "": "" n n "", "" v...",0
4,"{"" n i Id"": 656, "" n i "": "" n n "", "" v...",0
...,...,...
4757,"{"" n i Id"": 990, "" n i "": "" n n "", "" v...",3
4758,"{"" n i Id"": 991, "" n i "": "" n n "", "" v...",3
4759,"{"" n i Id"": 992, "" n i "": "" n n "", "" v...",3
4760,"{"" n i Id"": 993, "" n i "": "" n n "", "" v...",3


In [24]:
df['y'].value_counts()

1    2020
3    2019
2     431
0     292
Name: y, dtype: int64

# 14.2　特征提取

### 图片化

In [32]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# 如果使用自定义的save_image函数，确保已经定义或导入

def save_image(data,index):
    from PIL import Image
    new_im = Image.fromarray(data)
    new_im.save("../Data/%s.bmp" % index)

def get_feature_pe_picture():
    # 加载原始文件
    x, y = load_files()
    max_features = 1024
    vectorizer = CountVectorizer(
        decode_error='ignore',
        ngram_range=(2, 2),
        strip_accents='ascii',
        max_features=max_features,
        stop_words='english',
        max_df=1.0,
        min_df=1,
        token_pattern=r'\b\w+\b',
        binary=False)
    # Fit and transform the data
    x = vectorizer.fit_transform(x).toarray()
    x_pic = []
    for i in range(len(x)):  # 确保循环遍历所有数据
        pic = np.reshape(x[i], (32, 32, 1))
        x_pic.append(pic)
        if i==5:
            save_image(pic, i)  # 如果需要保存图像，取消注释并确保save_image已定义或导入

    # 随机分配训练和测试集合
    x_train, x_test, y_train, y_test = train_test_split(x_pic, y, test_size=0.4, random_state=42)  # 添加random_state以确保可重复性
    return x_train, x_test, y_train, y_test

# 调用函数
x_train, x_test, y_train, y_test = get_feature_pe_picture()

Load file d:\Documents\天池竞赛\WebSecMLLearnNote\Data\MalwareTrainingSets-master\trainingSets\APT1\652_anqcj.json
Load file d:\Documents\天池竞赛\WebSecMLLearnNote\Data\MalwareTrainingSets-master\trainingSets\APT1\653_mlchm.json
Load file d:\Documents\天池竞赛\WebSecMLLearnNote\Data\MalwareTrainingSets-master\trainingSets\APT1\654_ncuyn.json
Load file d:\Documents\天池竞赛\WebSecMLLearnNote\Data\MalwareTrainingSets-master\trainingSets\APT1\655_jnzct.json
Load file d:\Documents\天池竞赛\WebSecMLLearnNote\Data\MalwareTrainingSets-master\trainingSets\APT1\656_jxsmu.json
Load file d:\Documents\天池竞赛\WebSecMLLearnNote\Data\MalwareTrainingSets-master\trainingSets\APT1\657_yaeri.json
Load file d:\Documents\天池竞赛\WebSecMLLearnNote\Data\MalwareTrainingSets-master\trainingSets\APT1\658_hqyki.json
Load file d:\Documents\天池竞赛\WebSecMLLearnNote\Data\MalwareTrainingSets-master\trainingSets\APT1\659_lgumi.json
Load file d:\Documents\天池竞赛\WebSecMLLearnNote\Data\MalwareTrainingSets-master\trainingSets\APT1\660_urqsn.json
L

TypeError: Cannot handle this data type: (1, 1, 1), <i8

### tfidf

In [34]:
def get_feature_text():
    x,y=load_files()
    max_features=1000

    vectorizer = CountVectorizer(
            decode_error='ignore',
            ngram_range=(2, 2),
            strip_accents='ascii',
            max_features=max_features,
            stop_words='english',
            max_df=1.0,
            min_df=1,
            token_pattern=r'\b\w+\b',
            binary=True)
    # print vectorizer
    x=vectorizer.fit_transform(x)

    transformer = TfidfTransformer(smooth_idf=False)
    x = transformer.fit_transform(x)

    # 非常重要 稀疏矩阵转换成矩阵
    x = x.toarray()

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)
    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = get_feature_text()

Loaded files 4762


# 14.3　模型训练与验证
## 14.3.1　支持向量机算法

In [35]:

def do_svm(x_train, x_test, y_train, y_test):
    from sklearn.svm import SVC
    clf = svm.SVC(kernel='linear', C=1.0)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))
do_svm(x_train, x_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       116
           1       0.95      0.95      0.95       831
           2       0.90      0.90      0.90       179
           3       0.94      0.94      0.94       779

    accuracy                           0.94      1905
   macro avg       0.94      0.93      0.94      1905
weighted avg       0.94      0.94      0.94      1905



### 14.3.2　XGBoost算法

In [38]:

def do_xgboost(x_train, x_test, y_train, y_test):
    xgb_model = xgb.XGBClassifier().fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)
    print(classification_report(y_test, y_pred))

do_xgboost(x_train, x_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       116
           1       0.97      0.94      0.96       831
           2       0.97      0.94      0.95       179
           3       0.93      0.97      0.95       779

    accuracy                           0.95      1905
   macro avg       0.96      0.96      0.96      1905
weighted avg       0.96      0.95      0.95      1905



### 14.3.3　深度学习算法之多层感知机

In [39]:
def do_mlp(x_train, x_test, y_train, y_test):

    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes = (10, 4),
                        random_state = 1)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))
    #print metrics.confusion_matrix(y_test, y_pred)
do_mlp(x_train, x_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.50      0.91      0.64       116
           1       0.89      0.87      0.88       831
           2       0.68      0.29      0.41       179
           3       0.90      0.94      0.92       779

    accuracy                           0.84      1905
   macro avg       0.74      0.75      0.71      1905
weighted avg       0.85      0.84      0.84      1905



d:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


### CNN

In [ ]:
def do_cnn(trainX, testX, trainY, testY):
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=4)
    testY = to_categorical(testY, nb_classes=4)
    # Building convolutional network
    network = input_data(shape=[None, 32, 32,1], name='input')
    network = conv_2d(network, 16, 3, activation='relu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = conv_2d(network, 16, 3, activation='relu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = fully_connected(network, 16, activation='tanh')
    network = dropout(network, 0.1)
    network = fully_connected(network, 16, activation='tanh')
    network = dropout(network, 0.1)
    network = fully_connected(network, 4, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.01,
                         loss='categorical_crossentropy', name='target')

    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit(trainX, trainY, n_epoch=10, validation_set=(testX, testY),show_metric=True, run_id="malware")

In [40]:

def do_cnn_1d(trainX, testX, trainY, testY):

    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=4)
    testY = to_categorical(testY, nb_classes=4)

    # Building convolutional network
    network = input_data(shape=[None,1000], name='input')
    network = tflearn.embedding(network, input_dim=1000000, output_dim=128,validate_indices=False)
    branch1 = conv_1d(network, 128, 3, padding='valid', activation='relu', regularizer="L2")
    branch2 = conv_1d(network, 128, 4, padding='valid', activation='relu', regularizer="L2")
    branch3 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")
    network = merge([branch1, branch2, branch3], mode='concat', axis=1)
    network = tf.expand_dims(network, 2)
    network = global_max_pool(network)
    network = dropout(network, 0.8)
    network = fully_connected(network, 4, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')
    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit(trainX, trainY,
              n_epoch=5, shuffle=True, validation_set=(testX, testY),
              show_metric=True, batch_size=100,run_id="malware")
do_cnn_1d(x_train, x_test, y_train, y_test)

Training Step: 144  | total loss: 1.11774 | time: 78.723s
| Adam | epoch: 005 | loss: 1.11774 - acc: 0.4277 -- iter: 2800/2857
Training Step: 145  | total loss: 1.11116 | time: 91.750s
| Adam | epoch: 005 | loss: 1.11116 - acc: 0.4270 | val_loss: 1.12197 - val_acc: 0.4089 -- iter: 2857/2857
--
